In [1]:
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob
from pathlib import Path
from tqdm import tqdm

import cv2
import numpy as np

In [2]:
path = Path('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data')
im_path = path/'images'
labels = path/'labels.csv'

## Image parameters

### Mean distribution

### Median distribution

### Std distribution

## Image embedding

In [3]:
model = foz.load_zoo_model("resnet50-imagenet-torch")
print(model.has_embeddings)

/home/khoa-ys/Projects/auto_inspection/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/khoa-ys/Projects/auto_inspection/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


True


In [4]:
image_list = list(im_path.iterdir())
print(len(image_list))
image_list[:10]

2624


[PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell2346.png'),
 PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell2391.png'),
 PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell0837.png'),
 PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell0625.png'),
 PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell0682.png'),
 PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell2356.png'),
 PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell2262.png'),
 PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell2322.png'),
 PosixPath('/home/khoa-ys/Projects/auto_inspection/research.informationsystem.elpv/data/images/cell2541.png'),
 

In [5]:

dataset = fo.Dataset("dataset")
for img_path in image_list:
    dataset.add_sample(fo.Sample(filepath=str(img_path)))

#dataset = fo.load_dataset('dataset')
dataset.persistent = True
dataset

Name:        dataset
Media type:  image
Num samples: 2624
Persistent:  True
Tags:        []
Sample fields:
    id:       fiftyone.core.fields.ObjectIdField
    filepath: fiftyone.core.fields.StringField
    tags:     fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)

In [6]:
embeddings = dataset.compute_embeddings(model)

 100% |███████████████| 2624/2624 [22.8s elapsed, 0s remaining, 95.0 samples/s]      


In [7]:
results = fob.compute_visualization(
    dataset,
    embeddings=embeddings,
    num_dims=2,
    brain_key="image_embeddings",
    verbose=True,
    seed=51,
    method='umap'
)

Generating visualization...
UMAP(random_state=51, verbose=True)
Thu Sep  8 09:35:08 2022 Construct fuzzy simplicial set
Thu Sep  8 09:35:14 2022 Finding Nearest Neighbors
Thu Sep  8 09:35:16 2022 Finished Nearest Neighbor Search
Thu Sep  8 09:35:17 2022 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

Thu Sep  8 09:35:21 2022 Finished embedding


In [8]:
session = fo.launch_app(dataset)

In [9]:
plot = results.visualize()
plot.show(height=720)

session.plots.attach(plot)

FigureWidget({
    'data': [{'customdata': array(['63198cfb15e63a360113bde3', '63198cfb15e63a360113bde4',
    …